In [ ]:
import numpy as np
from section4_1_Q import get_Q_T_H_d_t_i

## 第六節　電気蓄熱暖房器

1．適用範囲～4．記号及び単位は省略

### 5．最大暖房出力

$$\large
Q_{max,H,d,t} = q_{rtd,H} \times 3600 \times 10^{-6}
$$

$Q_{max,H,d,t}$：日付$d$の時刻$t$における1時間当たりの最大暖房出力(MJ/h)  
$q_{rtd,H}$：定格暖房能力(W)

In [ ]:
def get_Q_max_H_d_t(q_rtd_H):
    return np.ones(24 * 365) * (q_rtd_H * 3600 / 1000000)  # (1)

### 6．暖房エネルギー消費量

#### 6.1　消費電力量

$$
\large
E_{E,H,d,t} = Q_{T,H,d,t} \times \frac{1}{e_{rtd,H}} \times 10^3 \div 3600
$$

$E_{E,H,d,t}$：日付$d$の時刻$t$における1時間当たりの消費電力量(kWh/h)  
$Q_{T,H,d,t}$：日付$d$の時刻$t$における1時間当たりの処理暖房負荷(MJ/h)  
$e_{rtd,H}$：蓄熱効率  

In [ ]:
def get_E_E_H_d_t(q_rtd_H, e_rtd_H, L_H_d_t):
    # 最大暖房出力
    Q_max_H_d_t = get_Q_max_H_d_t(q_rtd_H)

    # 処理暖房負荷
    Q_T_H_d_t = get_Q_T_H_d_t_i(Q_max_H_d_t, L_H_d_t, None, '居室のみを暖房する方式でかつ主たる居室とその他の居室ともに温水暖房を設置する場合に該当しない場合')

    return Q_T_H_d_t * (1.0 / e_rtd_H) * 1000 / 3600  # (2)


def get_Q_UT_H_d_t(q_rtd_H, e_rtd_H, L_H_d_t):
    # 最大暖房出力
    Q_max_H_d_t = get_Q_max_H_d_t(q_rtd_H)

    # 処理暖房負荷
    Q_T_H_d_t = get_Q_T_H_d_t_i(Q_max_H_d_t, L_H_d_t, None, '居室のみを暖房する方式でかつ主たる居室とその他の居室ともに温水暖房を設置する場合に該当しない場合')

    return L_H_d_t - Q_T_H_d_t

#### 6.2　ガス消費量

日付$d$の時刻$t$における１時間当たりのガス消費量$E_{G,H,d,t}$は0とする。

In [ ]:
def get_E_G_H_d_t():
    return np.zeros(24 * 365)

#### 6.3　灯油消費量

日付$d$の時刻$t$における１時間当たりの灯油消費量$E_{K,H,d,t}$は0とする。

In [ ]:
def get_E_K_H_d_t():
    return np.zeros(24 * 365)

#### 6.4　その他の燃料による一次エネルギー消費量

日付$d$の時刻$t$における１時間当たりのその他の燃料による一次エネルギー消費量$E_{M,H,d,t}$は0とする。

In [ ]:
def get_E_M_H_d_t():
    return np.zeros(24 * 365)

## 付録 A 機器の性能を表す仕様の決定方法

### A.1　記号及び単位

省略

### A.2　定格暖房能力

$$\large
q_{rtd,H} = q_{rq,H} \times A_{HCZ} \times f_{cT} \times f_{cI}
$$

$q_{rtd,H}$：定格暖房能力(kWh/h)  
$q_{rq,H}$：単位面積当たりの必要暖房能力(W/m<sup>2</sup>)  
$A_{HCZ}$：電気蓄熱暖房器により暖房される暖冷房区画の床面積(m<sup>2</sup>)  
$f_{cT}$：外気温度能力補正係数  
$f_{cI}$：間歇運転能力補正係数  

外気温度補正係数$f_{cT}$の値は1.05とする。

In [ ]:
def get_q_rtd_H(q_rq_H, A_HCZ, f_cT, f_cI):
    return q_rq_H * A_HCZ * f_cT * f_cI  # (1)


# 単位面積当たりの必要暖房能力
def get_q_rq_H(region):
    return table_a_2[region - 1]


# 外気温度補正係数
def get_f_cT(region):
    return 1.05


# 間歇運転能力補正係数
def get_f_cI(mode, R_type):
    if mode in ['ろ', '連続']:
        y = 0
    elif mode in ['は', '間歇']:
        y = 1
    else:
        raise ValueError(mode)

    if R_type == '主たる居室':
        x = 0
    elif R_type == 'その他の居室':
        x = 1
    else:
        raise ValueError(R_type)

    return table_a_3[y][x]

<div style="text-align: center;font-weight: bold;">表　単位面積当たりの必要暖房能力（W/m<sup>2</sup>）</div>

|地域の区分|1|2|3|4|5|6|7|8|
|-|-|-|-|-|-|-|-|-|
|　|139.3|120.7|111.3|119.0|126.6|106.5|112.9|-|

In [ ]:
# 表A.2 単位面積当たりの必要暖房能力
table_a_2 = [
    139.3,
    120.7,
    111.3,
    119.0,
    126.6,
    106.5,
    112.9
]

<div style="text-align: center;font-weight: bold;">表　間歇運転能力補正係数</div>

|　|主たる居室|その他の居室|
|-|-|-|
|連続運転の場合|1.0|1.0|
|間歇運転の場合 |3.034|4.805|

In [ ]:
# 表A.3 間歇運転能力補正係数
table_a_3 = [
    (1.0, 1.0),
    (3.034, 4.805)
]

### A.3　蓄熱効率 

蓄熱効率$e_{rtd.H}$は0.850（85.0%）とする。 

In [ ]:
def get_e_rtd_H():
    return 0.850